# Algorithme génétique

Ce programme est composé de plusieurs blocs segmentant l'éxécution du programme.

Le premier bloc est dédié à la définition des fonctions de l'algorithme qui seront par la suite appellées dans le programme. Ce bloc n'a besoin d'être éxécuté qu'une seule fois. Ces fonctions sont divisés en deux ensemble. 


La première partie comporte les fonctions nécessaires à la création  de notre situation d'origine :

    1) Création de n points au coordonées aléatoires représentant nos villes, ainsi que notre entrepot situé au mileu de la        plage de nombre aléatoire
    
    2) Affichage des points sur un graphe 
    
    3) Sélection parmis les villes de n points de livraison (points verts)
    
    4) Création d'une matrice d'adjacence aléatoire limitée par un nombre de voisin maximum pour chaque point. La matrice          est générée de tel manière à créer un seul graphe connexe. Grâce à cette matrice, nous obtenons une représentation          réaliste d'une carte routière. Chaque ville est connectée à quelques autres seulement et non à toute les autres            (notre graphe n'est pas complet). Ce choix rendra l'application de l'algorithme génétique bien plus complexe à              mettre en place au profit d'un respect du context initial bien plus proche de la réalité et nécessitera                    d'importantes modifications dans la méthode le l'algorithme. On peut donc considérer ce choix comme l'application          d'une contrainte supplémentaire autre que celles obligatoires.

    5) Traçage des routes entre les points selon la matrice.
    
    6) Applique une pondération à la matrice en fonction de la distance entre les points voisins, calculée à partir des            coordonées de chaque point.
<br>
    
    
La deuxième partie est dédié à la définitions des fonctions nécessaire à l'algorithme génétique.

    1) Génération d'une solution valide passant par tous les points de livraisons et retournant à l'entrepot.
    
    2) Calcul de la longeur d'une solution
    
    3) Génération d'une population de n solutions 
    
    4) Selection par tournoi d'une population. Les solutions sont regroupée par paire et on garde pour chaque paire la            meilleure des deux solutions. Pour n solutions en entrée, on retourne n/2 solutions.
    
    5) Mutation des solutions issues de la sélection. On retourne les solutions en entrée ainsi qu'une copie mutée de              chaque solution. La mutation ne mute pas simplement un gène aléatoire (un point dans la solution) mais nécessite de        regénérer toute la suite de la solution à partir de ce point car simplement substituer un point mènerai à une              incohérence car le nouveau point ne serai pas forcément voisin avec le point le précédant ainsi que le point                suivant. C'est pour cette raison qu'il n'existe pas d'étape de croisement dans cet algorithme. La substitution de          points entre deux solutions est impossible.

    6) Éxécution de la fonction de création de population puis de sélection et enfin de mutation. On sélectionne et mute n        fois en utilisant en entrée de la sélection de rang i+1 les solutions générées par la mutation de rang i. Pour              chaque itération, on retourne la solution la plus courte de la population.

    7) Trace la solution sur un graphe.

    8) Les trois dernière fonctions permettent d'identifier les allers et retours inutiles (ne passant pas par un point de        livraison) d'une solution et de les supprimer pour donner une solution encore plus pertinente. 

<br>
<br>
Le deuxième bloc nous permet d'initialiser les valeur paramètres de notre situation, comme le nombre de points, le nombre de voisins maximum, le nombre de colis ou bien le nombre d'itération et la population de l'algorithme génétique. Il va également lancer les fonctions de génération de la situation. Suite à son éxécution, le graphe de notre situation est retourné.

Le troisième bloc va lancer l'algorithme génétique à la recherche d'une solution. Nous retrouvons pour chaque itération la solution la plus courte ainsi que sa longeur. On obtient à la fin des itération, un graphe des longeurs des solutions en fonctions du nombre d'itération ainsi que le temps d'éxécution de l'algorithme.

Le quatrième bloc va tracer la solution trouvée.

Le cinquième bloc va chercher toute optimisation en supprimant les allers-retours inutiles et tracer le graphe de la solution finale.

<br>

Note: Les distances sont en kilomètres, on a simulé des points a grande distances, il peut donc y avoir deux points avec 700Km d'écart, nos temps de routes seront donc importants. De plus, une bibliographie sera disponible à la fin de ce document afin de mettre à disposition les sources sur lesquelles s'appliquent 

# Définition des fonctions 

In [ ]:
%matplotlib inline

import random
import numpy as np
import matplotlib.pyplot as plt


#------------------------------------------------------------------------------------------------------------------------
#Generating n points at random coordinates and return them in a list

def pointGeneration(nb,scaleMultiplier):
    n = nb
    scale = scaleMultiplier*n
    depot = (scale/2,scale/2)
    pointList = [depot]
    
    for i in range(n):
        pointList += (i,)
        pointList[i+1] = (random.randrange(0,scale),random.randrange(0,scale))
        
    return pointList

#------------------------------------------------------------------------------------------------------------------------
#Display all the points on the graph

def plotPoints(pointList):
    plt.plot(pointList[0][0],pointList[0][1],"ro", markersize=15)
    plt.annotate("dépot", (pointList[0][0]+3, pointList[0][1]+3), fontsize=10)
    
    for i in range(len(pointList)):
        if (i!= 0):
            plt.plot(pointList[i][0], pointList[i][1],"bo")
            plt.annotate(i, (pointList[i][0]+3, pointList[i][1]+3), fontsize=12)
            
#------------------------------------------------------------------------------------------------------------------------
#Choose n points as delivery points, display them on the graph and return them in a list

def randomPackages(n, nbPointsMax):
    package = random.sample(range(1, nbPointsMax), n)
    
    for i in range(len(package)):
        plt.plot(pointList[package[i]][0], pointList[package[i]][1],"go", markersize=10)
        
    return package

#------------------------------------------------------------------------------------------------------------------------
#Creating the adjacency matrix so the graph is connected and return the matrix

def makematrix(liste, linksLimit):
    matrix = np.zeros((len(liste),len(liste)))
    
    for i in range(len(matrix)):
        matrix[i][(i+1)%(len(matrix))] = 1
        matrix[(i+1)%(len(matrix))][i] = 1
    
    for i in range(len(matrix)-1):
        
            nbchoisi = 0
            while(sum(matrix[i])<linksLimit and matrix[:, i].sum() < linksLimit):
                nbchoisi = np.random.randint(i,len(matrix))
                
                if (matrix[:, nbchoisi].sum() < linksLimit and sum(matrix[nbchoisi])<linksLimit):
                    matrix[i][nbchoisi] = 1
                    matrix[nbchoisi][i] = 1
                   
    for i in range(len(matrix)):
        matrix[i][i] = 0  
    return matrix

#------------------------------------------------------------------------------------------------------------------------
#Display links between every neighbor points    

def TraceLinks(matrix):
    
    for i in range(len(matrix)):
        for j in range(len(matrix)):
            if(i>j):
                if matrix[i][j] > 0:
                    x = (pointList[i][0], pointList[j][0])
                    y = (pointList[i][1], pointList[j][1])
                    plt.plot(x, y,"b", linestyle="solid")
   
#------------------------------------------------------------------------------------------------------------------------
#Give weight to every links in the matrix depending on the coordinates difference of the points

def ponderation(pointList, matrix):
    
    for i in range(len(matrix)):
        for j in range(len(matrix)):
            if matrix[i][j] == 1:
                poids = np.sqrt((pointList[i][0]-pointList[j][0])**2+(pointList[i][1]-pointList[j][1])**2)
                matrix[i][j] = round(poids)
                
#------------------------------------------------------------------------------------------------------------------------

#------------------------------------------------------------------------------------------------------------------------
#Generate a random existing solution starting from the depot, including all delivery points and coming back

def generateSolution(matrix,package):

    packageLeft = package.copy()
    solution = [0]

    while(packageLeft):
        randomNext = np.random.randint(0,len(matrix))
        while( matrix[solution[-1]][randomNext] == 0 ):
            randomNext = np.random.randint(0,len(matrix))
        solution.append(randomNext)
        if(randomNext in packageLeft):
            packageLeft.remove(randomNext)
    
    while(solution[-1] != 0):
        randomNext = np.random.randint(0,len(matrix))
        while( matrix[solution[-1]][randomNext] == 0 ):
            randomNext = np.random.randint(0,len(matrix))
        solution.append(randomNext)
    
    return solution

#------------------------------------------------------------------------------------------------------------------------
#Return the length of the solution

def solutionLength(solution):
    
    totalLength = 0
    
    for i in range(0,len(solution)-1):
        totalLength += matrix[solution[i]][solution[i+1]]
        
    return totalLength

#------------------------------------------------------------------------------------------------------------------------
#Generate n random existing solutions and return them in a list

def generatePopulation(matrix,package,populationSize):

    population = []
    for i in range(populationSize):

        population.append(generateSolution(matrix,package))
        
    return population

#------------------------------------------------------------------------------------------------------------------------
#Return only the shortest half of solutions in a list

def selection(population):
    
    population2 = []
    
    for i in range(0,len(population)-1,2):
        if(solutionLength(population[i]) < solutionLength(population[i+1])):
            population2.append(population[i])
        else:
            population2.append(population[i+1])
            
    return population2
    
#------------------------------------------------------------------------------------------------------------------------
#Mutate the given population and return a new population composed of mutated + given population

def mutation(population2,matrix,package):
    
    mutatedPopu = []
    
    for i in range(len(population2)):
        mutatedSolu = population2[i].copy()
        mutatedPopu.append(population2[i].copy())
        mutationRate = np.random.randint(1,len(mutatedSolu)-1)
        del mutatedSolu[-mutationRate:]
        
        packageLeft = package.copy()

        for i in range(0,len(mutatedSolu)-1):
            if(mutatedSolu[i] in packageLeft):
                packageLeft.remove(mutatedSolu[i])

        while(packageLeft):
            randomNext = np.random.randint(0,len(matrix))
            while( matrix[mutatedSolu[-1]][randomNext] == 0 ):
                randomNext = np.random.randint(0,len(matrix))
            mutatedSolu.append(randomNext)
            if(randomNext in packageLeft):
                packageLeft.remove(randomNext)
            
        while(mutatedSolu[-1] != 0):
            randomNext = np.random.randint(0,len(matrix))
            while( matrix[mutatedSolu[-1]][randomNext] == 0 ):
                randomNext = np.random.randint(0,len(matrix))
            mutatedSolu.append(randomNext)

        mutatedPopu.append(mutatedSolu)
    
    return mutatedPopu

#------------------------------------------------------------------------------------------------------------------------
#Apply selection and mutation n times and return the shortest solution

def iterate(nbIteration, matrix, package, populationSize):

    initPopulation = generatePopulation(matrix, package, populationSize)
    select = selection(initPopulation)
    lastMin = None
    
    for i in range(nbIteration):
        
        length = []
        muta = mutation(select,matrix, package)
        select = selection(muta)
        
        for j in range(len(muta)):
            length.append(solutionLength(muta[j]))
            
            
        plt.plot([i,i+1], [lastMin,min(length)],"b", linestyle="solid",markersize=10)
        lastMin = min(length)
        
            
        print(i, ":",muta[length.index(min(length))])
        #print(i)
        #print("index :",length.index(min(length)))
        print("length :",min(length),"\n")
    
    return muta[length.index(min(length))]

#------------------------------------------------------------------------------------------------------------------------
#Display the solution on the graph

def traceSolution(s,package, pointList):
    plt.plot(pointList[0][0],pointList[0][1],"ro", markersize=15)
    plt.annotate("dépot", (pointList[0][0]+3, pointList[0][1]+3), fontsize=10)

    for i in range(len(package)):

        plt.plot(pointList[package[i]][0], pointList[package[i]][1],"go", markersize=10)

    for i in range(0,len(s)-1):
        x = (pointList[s[i]][0], pointList[s[i+1]][0])
        y = (pointList[s[i]][1], pointList[s[i+1]][1])
        plt.plot(x, y,"b", linestyle="solid")
        plt.annotate(s[i+1], (pointList[s[i+1]][0]+3, pointList[s[i+1]][1]+3), fontsize=12)

        
#------------------------------------------------------------------------------------------------------------------------
#Functions to remove uselsess back and forth in the solution of iterations

def verifPresenceElemtable1_In_table2(elemsAVerifier, TableauEntier):
    for e in elemsAVerifier:
        if(e in TableauEntier):
            return True
        
    return False  
    
    
def verif_Not_Useless_Path(solution, package):
    for i in solution:
        if solution.count(i) > 1:
            occurs = [j for j, x in enumerate(solution) if x == i]
            for o in range(len(occurs)):
                if (o != len(occurs)-1):
                    if (not(verifPresenceElemtable1_In_table2(package,solution[occurs[o]+1:occurs[o+1]+1]))):
                        del solution[occurs[o]+1:occurs[o+1]+1]
                        return False
    return True

#--------------------------------------------------------------------------------------------------------------------------------------------
            

def remove_useless_subpaths(solution, package):
    verif = False
    while(verif == False):#REMPLACER PAR WHILE
        verif = verif_Not_Useless_Path(solution, package)
        
    return solution


# Initialisation des paramètres et création de la situation

In [ ]:
nbPoints = 20
nbNeighbor = 3
nbPackage = 4
scaleMultiplier = 10

nbIteration = 10
populationSize = 50



pointList = pointGeneration(nbPoints,scaleMultiplier)
plotPoints(pointList)
#print("Liste des points :", pointList)

package = randomPackages(nbPackage, nbPoints)
#print("Liste des paquets :", package)

matrix = makematrix(pointList, nbNeighbor)
#print("Matrice d'adjacence \n", matrix)

ponderation (pointList,matrix)
#print("Matrice d'adjacence pondérée \n", matrix)

TraceLinks(matrix)


# Lancement de l'algorithme génétique

In [ ]:
%%time
s = iterate(nbIteration,matrix,package, populationSize)

# Affichage de la solution générée

In [ ]:
traceSolution(s,package,pointList)

# Optimisation de la solution générée

In [ ]:
print("Solution générée :",s)
print("Longueur :",solutionLength(s))

print("\nSolution optimisée :",remove_useless_subpaths(s, package))
print("Longueur :",solutionLength(remove_useless_subpaths(s, package)))

traceSolution(remove_useless_subpaths(s, package),package, pointList)

# Bibliographie

**Choice of best possible metaheuristic algorithm for the travelling salesman problem with limited computational time: quality, uncertainty and speed (2013)**
<br>
Auteurs : Marek Antosiewicz, Grzegorz Koloch, Bogumił Kamiński 
<br>
https://www.researchgate.net/profile/Bogumil-Kaminski-2/publication/312889331_Choice_of_best_possible_metaheuristic_algorithm_for_the_travelling_salesman_problem_with_limited_computational_time_Quality_uncertainty_and_speed/links/5fc22195a6fdcc6cc677a46a/Choice-of-best-possible-metaheuristic-algorithm-for-the-travelling-salesman-problem-with-limited-computational-time-Quality-uncertainty-and-speed.pdf
<br>
<br>
**An Improved Genetic Algorithm with Initial Population Strategy for Symmetric TSP (2015)**
<br>
Auteur : Yong Deng, Yang Liu, et Deyun Zhou1
<br>
https://www.hindawi.com/journals/mpe/2015/212794/
<br>
<br>
**Genetic Algorithm Performance with Different Selection Strategies in Solving TSP (2011)**
<br>
Auteurs : Noraini Mohd Razali, John Geraghty
<br>
http://www.iaeng.org/publication/WCE2011/WCE2011_pp1134-1139.pdf
